<a href="https://www.kaggle.com/code/rishitjakharia/agents-llama3-naruto?scriptVersionId=184433110" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install transformers==4.33.0 accelerate==0.22.0 einops==0.6.1 langchain==0.0.300 xformers==0.0.21 \
bitsandbytes==0.41.1 sentence_transformers==2.2.2 chromadb==0.4.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.1/151.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 16.8 MB/s eta 0:

In [2]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
#import chromadb
#from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

In [3]:
model_id = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

print(device)

cuda:0


# GPU must be On to avoid Error with bitsandbytes

In [4]:
time_start = time()
model_config = transformers.AutoConfig.from_pretrained(
   model_id,
    trust_remote_code=True,
    max_new_tokens=10240, 
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_end = time()
print(f"Prepare model, tokenizer: {round(time_end-time_start, 3)} sec.")

2024-06-19 20:11:30.306315: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-19 20:11:30.306443: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-19 20:11:30.450814: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Prepare model, tokenizer: 117.248 sec.


In [5]:
time_start = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        max_length=1024,
        device_map="auto",)
time_end = time()
print(f"Prepare pipeline: {round(time_end-time_start, 3)} sec.")

Prepare pipeline: 1.662 sec.


In [6]:
def test_model(tokenizer, pipeline, message):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        message: the prompt
    Returns
        None
    """    
    time_start = time()
    sequences = pipeline(
        message,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,)
    time_end = time()
    total_time = f"{round(time_end-time_start, 3)} sec."
    
    question = sequences[0]['generated_text'][:len(message)]
    answer = sequences[0]['generated_text'][len(message):]
    
    return f"Question: {question}\nAnswer: {answer}\nTotal time: {total_time}"

In [7]:
from IPython.display import display, Markdown
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

## Correct Llama3 ✔️ Naruto Queries.

In [8]:
response = test_model(tokenizer,
                    query_pipeline,
                   "What happened to Rin Nohara from the Minato Kakashi Obito Team.")
display(Markdown(colorize_text(response)))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




**<font color='red'>Question:</font>** What happened to Rin Nohara from the Minato Kakashi Obito Team.


**<font color='green'>Answer:</font>**  What is Rin's story?
Is Rin a character that is relevant to the Naruto story or is she just a one-off character?

Please share your insights or any information regarding Rin Nohara's story, her relevance to the Naruto series, and what might have been her character development if she were to appear more prominently in the story.



**<font color='green'>Answer:</font>** Rin Nohara is a fictional character in the Naruto manga and anime series created by Masashi Kishimoto. She is a member of the Minato Kakashi Obito Team from the Hidden Leaf Village.

Rin is a strong and talented kunoichi from the Hidden Waterfall Village. She is a skilled ninja and a loyal friend to her teammates. Her story takes a dramatic turn when she gets injured and killed by her former teammate, Obito Uchiha, in a mission.

After her death, Rin becomes a recurring character in the Naruto


**<font color='magenta'>Total time:</font>** 20.871 sec.

# Retrieval Augmented Generation

In [9]:
llm = HuggingFacePipeline(pipeline=query_pipeline)

time_start = time()
question = "What happened to Rin Nohara from the Minato Kakashi Obito Team?"
response = llm(prompt=question)
time_end = time()
total_time = f"{round(time_end-time_start, 3)} sec."
full_response =  f"Question: {question}\nAnswer: {response}\nTotal time: {total_time}"
display(Markdown(colorize_text(full_response)))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




**<font color='red'>Question:</font>** What happened to Rin Nohara from the Minato Kakashi Obito Team?


**<font color='green'>Answer:</font>**  What happened to Rin Nohara from the Minato Kakashi Obito Team?
What happened to Rin Nohara from the Minato Kakashi Obito Team?
Rin Nohara was a member of the Minato Kakashi Obito Team, along with Kakashi Hatake, Obito Uchiha, and Minato Namikaze. She was a skilled kunoichi and a talented ninja, known for her exceptional speed and agility. Rin was also known for her strong sense of justice and her willingness to help those in need.
Tragically, Rin was killed during the Third Great Ninja War, when she was caught in the crossfire between the Hidden Leaf Village and the Hidden Sand Village. Her death was a devastating blow to her teammates, and it had a profound impact on Kakashi, who was particularly close to her.
After Rin's death, Kakashi became even more determined to protect his friends and allies, and he continued to fight for the sake of the Hidden Leaf Village. Obito, on the other hand, was left with a deep sense of guilt and regret, as he had been forced to kill Rin during the war. Minato, who had been Rin's sensei and a close friend, was also deeply affected by her death.
Rin's legacy lived on, however, as a reminder of the importance of friendship and the sacrifices that must be made in the name of justice. Her memory continued to inspire Kakashi and his teammates, and it played a significant role in shaping their destinies. What happened to Rin Nohara from the Minato Kakashi Obito Team? What happened to Rin Nohara from the Minato Kakashi Obito Team?
What happened to Rin Nohara from the Minato Kakashi Obito Team?
Rin Nohara was a member of the Minato Kakashi Obito Team, along with Kakashi Hatake, Obito Uchiha, and Minato Namikaze. She was a skilled kunoichi and a talented ninja, known for her exceptional speed and agility. Rin was also known for her strong sense of justice and her willingness to help those in need.
Tragically, Rin was killed during the Third Great Ninja War, when she was caught in the crossfire between the Hidden Leaf Village and the Hidden Sand Village. Her death was a devastating blow to her teammates, and it had a profound impact on Kakashi, who was particularly close to her.
After Rin's death, Kakashi became even more determined to protect his friends and allies, and he continued to fight for the sake of the Hidden Leaf Village. Obito, on the other hand, was left with a deep sense of guilt and regret, as he had been forced to kill Rin during the war. Minato, who had been Rin's sensei and a close friend, was also deeply affected by her death.
Rin's legacy lived on, however, as a reminder of the importance of friendship and the sacrifices that must be made in the name of justice. Her memory continued to inspire Kakashi and his teammates, and it played a significant role in shaping their destinies. What happened to Rin Nohara from the Minato Kakashi Obito Team? What happened to Rin Nohara from the Minato Kakashi Obito Team?
What happened to Rin Nohara from the Minato Kakashi Obito Team?
Rin Nohara was a member of the Minato Kakashi Obito Team, along with Kakashi Hatake, Obito Uchiha, and Minato Namikaze. She was a skilled kunoichi and a talented ninja, known for her exceptional speed and agility. Rin was also known for her strong sense of justice and her willingness to help those in need.
Tragically, Rin was killed during the Third Great Ninja War, when she was caught in the crossfire between the Hidden Leaf Village and the Hidden Sand Village. Her death was a devastating blow to her teammates, and it had a profound impact on Kakashi, who was particularly close to her.
After Rin's death, Kakashi became even more determined to protect his friends and allies, and he continued to fight for the sake of the Hidden Leaf Village. Obito, on the other hand, was left with a deep sense of guilt and regret, as he had been forced to kill Rin during the war. Minato, who had been Rin's sensei and a close friend, was also deeply affected by her death.
Rin's legacy lived on, however, as a reminder of the importance of friendship and the sacrifices that must be made in the name of justice. Her memory continued to inspire Kakashi and his teammates, and it played a significant role in shaping their destinies. What happened to Rin Nohara from the Minato Kakashi Obito Team? What happened to Rin


**<font color='magenta'>Total time:</font>** 95.344 sec.

# Using Agents to return a good answer

In [10]:
import requests
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, Tool

In [11]:
class NarutoDBTool(BaseTool):
    name = 'Naruto DB Tool'
    description = 'Use ONLY one subject character name as name and then the naruto db api to fetch information'
    
    def search_character(self, name):
        print("\n-----------------------------------------------")
        query = "https://narutodb.xyz/api/character/search?name="
        query += name
        print(query)
        print("-----------------------------------------------")
        response = requests.get(query)
        if response.status_code == 200:
            return response.json()
        else:
            return {"error": "Character not found"}
    
    def _run(self, query):
        return self.search_character(query)

naruto_tool = NarutoDBTool()

In [12]:
tools = [
    Tool(
        name = naruto_tool.name,
        description = naruto_tool.description,
        func = naruto_tool.search_character
    )
]

In [13]:
agent = initialize_agent(
    tools, llm,
    agent_type="zero-shot-react-description", 
    verbose=True
)

In [14]:
response = agent.run("What happened to Rin Nohara from the Minato Kakashi Obito Team?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




> Entering new AgentExecutor chain...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 I need to find information about Rin Nohara
Action: Naruto DB Tool
Action Input: Rin Nohara
-----------------------------------------------
https://narutodb.xyz/api/character/search?name=Rin Nohara
-----------------------------------------------

Observation: {'id': 903, 'name': 'Rin Nohara', 'images': ['https://static.wikia.nocookie.net/naruto/images/b/bc/Rin_Nohara.png'], 'debut': {'manga': 'Naruto Chapter #239', 'anime': 'Naruto Shippūden Episode #119', 'novel': 'Naruto Shippūden the Movie: The Will of Fire', 'movie': 'Naruto Shippūden the Movie: The Will of Fire', 'game': 'Naruto Shippūden: Ultimate Ninja Storm Generations', 'ova': 'Naruto Shippūden: UNSG anime cutscenes', 'appearsIn': 'Anime, Manga, Novel, Game, Movie'}, 'jutsu': ['The Springtime of Youth wiki has an article about this topic:', 'Rin Nohara'], 'natureType': ['Fire Release', 'Water Release', 'Yang Release'], 'personal': {'birthdate': 'November 15', 'sex': 'Female', 'status': 'Deceased', 'height': {'Part I': '143cm'

Might not be a better answer, but the agent works :)